In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt   
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

In [2]:
def geo_mean(iterable):
    a = np.array(iterable)
    return a.prod()**(1.0/len(a))

def sum_day(srs):
    number = (wt_srs*srs/(wt_srs*srs.notna()).sum()).sum()
    return number

In [3]:
df = pd.read_csv(r'~/PycharmProjects/ane_django/parsed_content/filled.csv', index_col='id').drop(columns='level_0')
df.head(1)

/Applications/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,category_id,category_title,date,miss,price_old,site_code,site_link,site_title,site_unit,type,price_new,nsprice_f
id,,,,,,,,,,,,
0,10,"Молоко питьевое цельное пастеризованное 2,5-3,...",2019-02-05,0,-1.0,globus,https://online.globus.ru/products/moloko-36-ko...,"Молоко пастеризованное 36 Копеек 3,2%, 485 мл",за 1 шт.,food,34.89,34.89


In [4]:
df = df[df.category_id!=76]
# df = df[df.site_code!='ozon']
df.loc[:,'unq'] = df.category_id.astype(str) + df.site_link
df.loc[:,'date'] = pd.to_datetime(df.date)
df = df[df.miss==0]
df = df.drop(columns=['category_title','site_title','site_unit',
                      'site_link','price_new','miss','price_old','site_code'])
df.head(1)

,category_id,date,type,nsprice_f,unq
id,,,,,
0,10,2019-02-05,food,34.89,10https://online.globus.ru/products/moloko-36-...


In [5]:
# type_ = 'non-food'
# df = df[df.type==type_]
srs = df.groupby('unq')['nsprice_f'].transform(lambda x: x.pct_change()+1)
df.loc[:,'cng_price'] = srs
# df = df.drop(columns=['nsprice_f'])

pvt_df = df.pivot_table(index='date', columns='category_id', aggfunc=geo_mean).loc[:,'cng_price'].iloc[1:]
pvt_df.iloc[0,:] = 1
pvt_df = pvt_df.fillna(method='ffill').apply(lambda x: x.cumprod(), axis=0).dropna(how='all')

In [6]:
wt_srs = pd.read_csv(r'~/PycharmProjects/ane_django/parser_app/logic/description/categories.csv', index_col='id', sep=';').iloc[:-1,:]['cpi_weight']
wt_srs.index = wt_srs.index.astype(int)
cpi_online = pvt_df.apply(lambda x: sum_day(x), axis=1)

In [7]:
cpi_of = pd.read_csv(f'~/cpi_common.csv',index_col='date', sep=';')
cpi_of.index = pd.to_datetime(cpi_of.index,format='%d.%m.%Y')
cpi_of = (cpi_of/100)
cpi_of = cpi_of.iloc[1:]
cpi_of.iloc[0] = 1
cpi_of = cpi_of.cumprod()
date_srs = pd.Series(index=pd.date_range(start=cpi_online.index[0], end=cpi_online.index[-1])).to_frame().drop(columns=0)
cpi_of = cpi_of.merge(date_srs, left_index=True, right_index=True, how='right').fillna(method='ffill')